In [ ]:
import pandas as pd
from IPython.display import display

# === Load Phase 3.5 snapshot and master ===
df_35 = pd.read_csv("drift/positions_2025-07-26_17-11-58.csv")
df_master = pd.read_csv("active_master.csv")

# === Focused Field Audit (3.5 → Master) ===
def audit_35_to_master(df_35, df_master):
    tracked_fields = [
        "TradeID", "Strategy", "Structure", "PCS", "PCS_Entry",
        "Delta_Entry", "Gamma_Entry", "Theta_Entry", "Vega_Entry",
        "Premium_Entry", "IVHV_Gap_Entry", "Skew_Entry", "Kurtosis_Entry",
        "Tag_Intent", "Tag_ExitStyle", "Confidence_Entry", "Confidence Tier",
        "BreakEven", "Capital Deployed", "Margin Required", "CostBasis_Entry",
        "Time_Val_Entry", "Intrinsic_Val_Entry", "TradeDate",
        "Earnings_Date", "Days_to_Earnings", "Is_Event_Setup"
    ]

    results = []
    for col in sorted(set(tracked_fields)):
        results.append({
            "Field": col,
            "In Phase 3.5": col in df_35.columns,
            "In Master": col in df_master.columns,
            "Nulls in Master": df_master[col].isnull().sum() if col in df_master.columns else None,
            "% Null in Master": round(df_master[col].isnull().mean() * 100, 2) if col in df_master.columns else None
        })

    audit_df = pd.DataFrame(results).sort_values("Field").reset_index(drop=True)
    display(audit_df)
    return audit_df

# === Run it
audit_df = audit_35_to_master(df_35, df_master)
audit_df.to_csv("audit_35_to_master.csv", index=False)


In [ ]:
import pandas as pd

df_test = pd.DataFrame([{
    "TradeID": "ABC123",
    "Delta": 0.52,
    "Gamma": 0.03,
    "Theta": -0.12,
    "Vega": 0.44,
    "PCS": 72,
    "Delta_Entry": 0.51,
    "Gamma_Entry": 0.03,
    "Theta_Entry": -0.11,
    "Vega_Entry": 0.42,
    "PCS_Entry": 71,
    "Premium": 1.25,
    "IVHV_Gap": 6.0,
    "Skew": 0.12,
    "Kurtosis": 2.8,
    "Strategy": "Buy Call"
}])

print("✅ df_test columns:\n", df_test.columns.tolist())
print("✅ Any missing in core freeze? →", set(["Delta", "Gamma", "Theta", "Vega", "PCS"]) - set(df_test.columns))


In [ ]:
import sys
sys.path.append("/Users/haniabadi/Documents/Github/options")

import os
print("core exists:", os.path.isdir("core"))
print("phase3_5_freeze_fields.py exists:", os.path.isfile("core/phase3_5_freeze_fields.py"))
print("phase3_5_freeze has init:", os.path.isfile("core/phase3_5_freeze/__init__.py"))

from core.phase3_5_freeze_fields import phase3_5_fill_freeze_fields
df_frozen = phase3_5_fill_freeze_fields(df_test.copy())
df_frozen[["Delta", "Delta_Entry", "Gamma", "Gamma_Entry", "PCS", "PCS_Entry", "BreakEven"]]



In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
from datetime import datetime
# import matplotlib.pyplot as plt

# 📂 Load master
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df = pd.read_csv(master_path, parse_dates=["TradeDate"], dayfirst=True, infer_datetime_format=True)

# 🧮 Compute Days_Held
today = pd.to_datetime(datetime.now().date())
df["Days_Held"] = (today - pd.to_datetime(df["TradeDate"], errors="coerce")).dt.days

# 🧪 Test 1: Freeze Integrity – Missing entry columns
freeze_cols = ["PCS_Entry", "Delta_Entry", "Vega_Entry", "Gamma_Entry", "Theta_Entry"]
for col in freeze_cols:
    missing = df[col].isnull().sum()
    print(f"❌ Missing {col}: {missing} rows" if missing else f"✅ {col} fully frozen.")

# 🧪 Test 2: Drift on new trades
fresh_drift = df[(df["Days_Held"] <= 1) & (df["PCS"] != df["PCS_Entry"])]
print(f"\n🚨 {len(fresh_drift)} fresh trades show PCS drift! (Should be zero)")

# 🧪 Test 3: Premature Exit or Revalidation flags
bad_flags = df[(df["Days_Held"] <= 1) & df["Rec_Action"].isin(["EXIT", "REVALIDATE", "TRIM"])]
print(f"\n🚨 {len(bad_flags)} trades flagged too early for action. Review logic.")

# # 📊 Optional Visualization
# df.plot.scatter(x="Days_Held", y="PCS_Drift", title="PCS Drift vs Days Held", alpha=0.7)
# plt.axhline(y=0, color='gray', linestyle='--')
# plt.grid(True)
# plt.show()


In [ ]:
# ✅ Imports
import pandas as pd
from datetime import datetime

# === ✅ Step 1: Load your latest master file ===
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"  # ← ✅ Update if needed
df = pd.read_csv(master_path)

# === ✅ Step 2: Robustly extract trade date info ===
# Check if these columns exist
df["Parsed_TradeDate"] = pd.to_datetime(df["TradeDate"], errors="coerce")

if "trade_date" in df.columns:
    df["Parsed_trade_date_raw"] = pd.to_datetime(df["trade_date"], unit="ms", errors="coerce")
else:
    df["Parsed_trade_date_raw"] = pd.NaT

# === ✅ Step 3: Freeze logic resolution
df["Final_TradeDate"] = df["Parsed_TradeDate"].combine_first(df["Parsed_trade_date_raw"])
df["Final_TradeDate"] = df["Final_TradeDate"].fillna(pd.to_datetime(datetime.now().date()))

# === ✅ Step 4: Calculate correct Days_Held
today = pd.to_datetime(datetime.now().date())
df["Correct_Days_Held"] = (today - df["Final_TradeDate"]).dt.days.clip(lower=1)

# === ✅ Step 5: Show audit
df_debug = df[[
    "TradeID", "TradeDate", "Parsed_TradeDate",
    "Parsed_trade_date_raw", "Final_TradeDate", "Correct_Days_Held",
    "Days_Held" if "Days_Held" in df.columns else df.columns[0]  # dummy to prevent crash
]]

# Display in notebook
df_debug.sort_values("Final_TradeDate", ascending=False).head(10)


In [ ]:
import os
import pandas as pd

# === Settings ===
DATA_DIR = "/Users/haniabadi/Documents/Windows/Optionrec/drift/"  # or "drift" if local
contracts_to_match = ["BKNG250801P5670", "BKNG250801C5730", "PLTR250822C150", "PLTR250822P150"]

# === 1. Collect All Matched Rows ===
snapshot_rows = []
for fname in sorted(os.listdir(DATA_DIR)):
    if fname.startswith("positions_") and fname.endswith(".csv"):
        path = os.path.join(DATA_DIR, fname)
        try:
            df = pd.read_csv(path)
            df["Snapshot_File"] = fname
            df["Snapshot_TS"] = pd.to_datetime(
                fname.replace("positions_", "").replace(".csv", ""),
                format="%Y-%m-%d_%H-%M-%S"
            )
            match = df[df["TradeID"].isin(contracts_to_match)].copy()
            if not match.empty:
                snapshot_rows.append(match)
        except Exception as e:
            print(f"⚠️ Error reading {fname}: {e}")

# Combine into single DataFrame
df_contract_ts = pd.concat(snapshot_rows, ignore_index=True) if snapshot_rows else pd.DataFrame()
print(f"✅ Found {len(df_contract_ts)} rows across snapshots")

# === 2. Compute D1/D3/D5 Drift Per Trade ===
drift_output = []
for tid, group in df_contract_ts.groupby("TradeID"):
    group = group.sort_values("Snapshot_TS").copy()
    group["Delta_1D"] = group["Delta"].diff(1)
    group["Gamma_3D"] = group["Gamma"].diff(3)
    group["Vega_5D"] = group["Vega"].diff(5)
    drift_output.append(group)

df_drifted = pd.concat(drift_output, ignore_index=True)
df_drifted = df_drifted.sort_values(["TradeID", "Snapshot_TS"])

# === 3. Preview or Export ===
from IPython.display import display
display(df_drifted[["TradeID", "Snapshot_TS", "Delta", "Gamma", "Vega", "Delta_1D", "Gamma_3D", "Vega_5D"]])


In [ ]:
import os
import pandas as pd

# === Settings
DATA_DIR = "/Users/haniabadi/Documents/Windows/Optionrec/drift/"
symbols_to_match = [
    "BKNG250801P5670", "BKNG250801C5730",
    "PLTR250822C150", "PLTR250822P150"
]

snapshot_rows = []

for fname in sorted(os.listdir(DATA_DIR)):
    if fname.endswith(".csv"):
        path = os.path.join(DATA_DIR, fname)
        try:
            df = pd.read_csv(path)
            if "Symbol" not in df.columns:
                continue
            df["Snapshot_File"] = fname

            # Try to extract timestamp from filename
            try:
                ts = fname.replace("positions_", "").replace(".csv", "")
                df["Snapshot_TS"] = pd.to_datetime(ts, format="%Y-%m-%d_%H-%M-%S")
            except:
                df["Snapshot_TS"] = pd.NaT

            matched = df[df["Symbol"].isin(symbols_to_match)].copy()
            if not matched.empty:
                snapshot_rows.append(matched)

        except Exception as e:
            print(f"⚠️ Error reading {fname}: {e}")

# Combine and sort
df_symbol_history = pd.concat(snapshot_rows, ignore_index=True) if snapshot_rows else pd.DataFrame()
df_symbol_history = df_symbol_history.sort_values(["Symbol", "Snapshot_TS"])

print(f"✅ Loaded {len(df_symbol_history)} rows for {len(set(df_symbol_history['Symbol']))} symbols.")
df_symbol_history[["Symbol", "Snapshot_File", "Snapshot_TS"]].head()

# === Compute Drift Metrics Per Symbol ===
drifted_rows = []

for symbol, group in df_symbol_history.groupby("Symbol"):
    group = group.sort_values("Snapshot_TS").copy()
    if "Delta" in group.columns:
        group["Delta_1D"] = group["Delta"].diff(1)
    if "Gamma" in group.columns:
        group["Gamma_3D"] = group["Gamma"].diff(3)
    if "Vega" in group.columns:
        group["Vega_5D"] = group["Vega"].diff(5)
    drifted_rows.append(group)

df_drift_final = pd.concat(drifted_rows, ignore_index=True)

# === Preview output
df_drift_final[["Symbol", "Snapshot_TS", "Delta", "Delta_1D", "Gamma", "Gamma_3D", "Vega", "Vega_5D"]].tail(10)

# === Step 1: Remove NaT and duplicate timestamps
df_cleaned = (
    df_symbol_history[df_symbol_history["Snapshot_TS"].notnull()]
    .drop_duplicates(subset=["Symbol", "Snapshot_TS"])
    .sort_values(["Symbol", "Snapshot_TS"])
    .reset_index(drop=True)
)

# === Step 2: Compute Drift per symbol
drifted_rows = []

for symbol, group in df_cleaned.groupby("Symbol"):
    group = group.copy()
    group["Delta_1D"] = group["Delta"].diff(1)
    group["Gamma_3D"] = group["Gamma"].diff(3)
    group["Vega_5D"] = group["Vega"].diff(5)
    drifted_rows.append(group)

df_drift_final = pd.concat(drifted_rows, ignore_index=True)

# === Preview: Cleaned Drift Table
df_drift_final[["Symbol", "Snapshot_TS", "Delta", "Delta_1D", "Gamma", "Gamma_3D", "Vega", "Vega_5D"]].tail(10)


# 1. Clean + drop duplicate timestamps
df_cleaned = (
    df_symbol_history[df_symbol_history["Snapshot_TS"].notnull()]
    .drop_duplicates(subset=["Symbol", "Snapshot_TS"])
    .sort_values(["Symbol", "Snapshot_TS"])
    .reset_index(drop=True)
)

# 2. Compute raw drift
drifted_rows = []

for symbol, group in df_cleaned.groupby("Symbol"):
    group = group.copy()
    group["Delta_1D"] = group["Delta"].diff(1)
    group["Gamma_3D"] = group["Gamma"].diff(3)
    group["Vega_5D"] = group["Vega"].diff(5)

    # Only keep rows where there's measurable drift
    drift_mask = (
        (group["Delta_1D"].abs() > 1e-5) |
        (group["Gamma_3D"].abs() > 1e-5) |
        (group["Vega_5D"].abs() > 1e-5)
    )
    group = group[drift_mask]
    drifted_rows.append(group)

# 3. Combine final drift-only rows
df_drift_filtered = pd.concat(drifted_rows, ignore_index=True)

# 4. Preview
df_drift_filtered[["Symbol", "Snapshot_TS", "Delta_1D", "Gamma_3D", "Vega_5D"]].sort_values("Snapshot_TS").tail(10)

# Step 1: Clean
df_cleaned = (
    df_symbol_history[df_symbol_history["Snapshot_TS"].notnull()]
    .drop_duplicates(subset=["Symbol", "Snapshot_TS"])
    .sort_values(["Symbol", "Snapshot_TS"])
    .reset_index(drop=True)
)

# Step 2: Compute + Retain Only the Final Valid Drift Row per Symbol
final_drift_rows = []

for symbol, group in df_cleaned.groupby("Symbol"):
    group = group.copy()
    group["Delta_1D"] = group["Delta"].diff(1)
    group["Gamma_3D"] = group["Gamma"].diff(3)
    group["Vega_5D"] = group["Vega"].diff(5)

    # ⛳ Keep only the row where all 3 are valid (last possible row in series)
    filtered = group[
        group["Delta_1D"].notnull() &
        group["Gamma_3D"].notnull() &
        group["Vega_5D"].notnull()
    ]

    # ✅ Keep only the latest drift row
    if not filtered.empty:
        final_drift_rows.append(filtered.iloc[[-1]])

# Step 3: Combine Final Result
df_drift_final_trimmed = pd.concat(final_drift_rows, ignore_index=True)

# Step 4: Preview
df_drift_final_trimmed[["Symbol", "Snapshot_TS", "Delta_1D", "Gamma_3D", "Vega_5D"]]


In [ ]:
# 📊 Phase Audit Utility: Snapshot → Master → Drift
import pandas as pd
from IPython.display import display

def audit_field_flow(df_35: pd.DataFrame, df_master: pd.DataFrame, df_drift: pd.DataFrame) -> pd.DataFrame:
    """
    Compare all important option fields across:
    - Phase 3.5 enriched snapshot
    - Master file (post Phase 6/6.5)
    - Drift snapshot (post Phase 7)
    
    Returns a dataframe showing presence and nulls.
    """
    core_fields = [
        # Freeze Metrics
        "Delta_Entry", "Gamma_Entry", "Vega_Entry", "Theta_Entry", "PCS_Entry",
        "IVHV_Gap_Entry", "Skew_Entry", "Kurtosis_Entry", "IV_Entry",
        "Premium_Entry", "CostBasis_Entry", "BreakEven", "Time_Val_Entry", "Intrinsic_Val_Entry",
        "Confidence_Entry", "Confidence Tier", "Structure", "Tag_Intent", "Tag_ExitStyle",
        "Margin Required", "Capital Deployed",

        # Lifecycle + Earnings
        "TradeDate", "Entry_Timestamp", "Earnings_Date", "Days_to_Earnings", "Is_Event_Setup",

        # Drift Snapshot Metrics
        "Delta_Drift", "Gamma_Drift", "Theta_Drift", "Vega_Drift", "PCS_Drift",
        "Delta_1D", "Gamma_3D", "Vega_5D",
        "Delta_1D_SMA", "Gamma_3D_SMA", "Vega_5D_SMA",
        "Delta_ROC", "Gamma_ROC", "Vega_ROC", "Theta_ROC",
        "Delta_ROC_percent", "Gamma_ROC_percent", "Vega_ROC_percent", "Theta_ROC_percent"
    ]

    audit = []
    for col in sorted(set(core_fields)):
        audit.append({
            "Field": col,
            "In Phase 3.5": col in df_35.columns,
            "In Master": col in df_master.columns,
            "In Drift Snapshot": col in df_drift.columns,
            "Nulls in Master": df_master[col].isnull().sum() if col in df_master.columns else None,
            "% Null in Master": round(df_master[col].isnull().mean() * 100, 2) if col in df_master.columns else None
        })

    audit_df = pd.DataFrame(audit)
    display(audit_df.sort_values("Field").reset_index(drop=True))
    return audit_df

audit_df = audit_pipeline_flow(df_35, df_master, df_drift)
display(audit_df[audit_df["Missing Anywhere"] == True])


In [ ]:
import pandas as pd
from IPython.display import display

# === 1. Load your three dataframes (edit paths if needed)
df_35 = pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/drift/positions_2025-07-26_17-11-58.csv")
df_master = pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv")
df_drift = pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/drift/drift_audit_2025-07-26_08-25-08.csv")

# === 2. Define audit function
def audit_field_propagation(df_35: pd.DataFrame, df_master: pd.DataFrame, df_drift: pd.DataFrame) -> pd.DataFrame:
    fields_to_track = [
        "PCS_Entry", "Delta_Entry", "Gamma_Entry", "Vega_Entry", "Theta_Entry",
        "IVHV_Gap_Entry", "Skew_Entry", "Kurtosis_Entry", "IV_Entry",
        "BreakEven", "Premium_Entry", "CostBasis_Entry", "Margin Required", "Capital Deployed",
        "Time_Val_Entry", "Intrinsic_Val_Entry",
        "Confidence_Entry", "Confidence Tier", "Structure", "Tag_Intent", "Tag_ExitStyle",
        "Delta_Drift", "Gamma_Drift", "PCS_Drift",
        "Delta_1D", "Gamma_3D", "Vega_5D",
        "Delta_1D_SMA", "Gamma_3D_SMA", "Vega_5D_SMA",
        "Delta_ROC", "Delta_ROC_percent", "Gamma_ROC_percent", "Theta_ROC_percent",
        "Earnings_Date", "Days_to_Earnings", "Is_Event_Setup",
        "TradeDate", "Days_Held", "Entry_Timestamp"
    ]

    rows = []
    for col in fields_to_track:
        rows.append({
            "Field": col,
            "In Phase 3.5": col in df_35.columns,
            "In Master": col in df_master.columns,
            "In Drift Snapshot": col in df_drift.columns,
            "Nulls in Master": df_master[col].isnull().sum() if col in df_master.columns else None,
        })

    audit_df = pd.DataFrame(rows)
    display(audit_df.sort_values("Field").reset_index(drop=True))

# === 3. Run it
audit_field_propagation(df_35, df_master, df_drift)


In [10]:
# === Phase 1: Load & Clean Raw Trades
from core.phase1_clean import phase1_load_and_clean_raw_v2
df, _ = phase1_load_and_clean_raw_v2()

# === Phase 2: Parse OCC Symbol + Tag Strategy
from core.phase2_parse import phase2_parse_symbols, phase21_strategy_tagging

df = phase2_parse_symbols(df)
df = phase21_strategy_tagging(df)


# === Phase 3: Calculate PCS + Greeks
from core.phase3_pcs_score import calculate_ivhv_gap, calculate_skew_and_kurtosis, calculate_pcs
df = calculate_ivhv_gap(df)
df = calculate_skew_and_kurtosis(df)
df = calculate_pcs(df)

# === Phase 3.5: Add PCS_Entry, Vega_Entry, TradeDate etc.
from core.phase3_5_freeze_fields import phase3_5_fill_freeze_fields
df = phase3_5_fill_freeze_fields(df)


# === Phase 4: Save to snapshot folder
from core.phase4_snapshot import phase4_save_snapshot
_, snapshot_path = phase4_save_snapshot(df)

# === Phase 5/6: Load Master + Archive Clean
from utils.load_master_snapshot import load_master_snapshot
from core.phase6_freeze_and_archive import phase6_freeze_and_archive
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df_master_current = load_master_snapshot(master_path)
df_master = phase6_freeze_and_archive(df, df_master_current)

# === Phase 6.5: Inject Derived Fields (e.g., ROI, Delta_Tiers)
from core.phase6_5 import phase6_5_inject_derived_fields
df_master = phase6_5_inject_derived_fields(df_master, save_path=master_path)

# from core.phase7_drift_engine import run_phase7_drift_engine
# drift_dir = "/Users/haniabadi/Documents/Windows/Optionrec/drift"
# df_drift = run_phase7_drift_engine(drift_dir=drift_dir, update_master=True)


# from core.chart_engine import run_phase8_chart_engine
# run_phase8_chart_engine(master_path=master_path)


⏳ Loading file: /Users/haniabadi/Documents/Windows/Positions_Account_.csv
🟢 File loaded!
🧹 Deduplicated 0 rows (Symbol only — fallback)
✅ Phase 1 complete.
📊 Rows: 7, Columns: 38
💾 Saved snapshot: /Users/haniabadi/Documents/Windows/Optionrec/drift/master_with_drift_2025-07-28_19-06-53.csv
⏱ Duration: 0.01s
✅ Phase 2.0 complete: OCC symbols parsed
✅ Phase 2.1 complete: Strategy tagging with TradeID-only logic


/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has 

📉 Skew nulls: 7 | TradeIDs: 7
🩹 Applied fallback: Skew/Kurtosis set to 0 where missing
📌 Phase 3.5: Running modular freeze pipeline...
🔍 Checking Delta → entry_col = Delta_Entry
    In df: True / True
    Head:     Delta  Delta_Entry
0  0.9612       0.9612
1 -0.5564      -0.5564
2  0.3783       0.3783
3 -0.3966      -0.3966
4  0.6080       0.6080
🔍 Checking Gamma → entry_col = Gamma_Entry
    In df: True / True
    Head:     Gamma  Gamma_Entry
0  0.0059       0.0059
1  0.0011       0.0011
2  0.0011       0.0011
3  0.0204       0.0204
4  0.0200       0.0200
🔍 Checking Theta → entry_col = Theta_Entry
    In df: True / True
    Head:      Theta  Theta_Entry
0  -0.1164      -0.1164
1 -17.7819     -17.7819
2 -16.7811     -16.7811
3  -0.1485      -0.1485
4  -0.1453      -0.1453
🔍 Checking Vega → entry_col = Vega_Entry
    In df: True / True
    Head:      Vega  Vega_Entry
0  0.0405      0.0405
1  2.3783      2.3783
2  2.2908      2.2908
3  0.2719      0.2719
4  0.2716      0.2716
🔍 Checking 

/Users/haniabadi/Documents/Github/options/core/phase3_5_freeze/freeze_breakeven.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(BreakEven=compute_group_breakeven(g)))


,TradeID,Delta_Entry,Gamma_Entry,Vega_Entry,Theta_Entry,IV_Entry
0,CAT250801C410,0.9612,0.0059,0.0405,-0.1164,25.09
1,BKNG250801P5670,-0.5564,0.0011,2.3783,-17.7819,58.92
2,BKNG250801C5730,0.3783,0.0011,2.2908,-16.7811,57.28
3,CRM250822P265,-0.3966,0.0204,0.2719,-0.1485,27.05
4,CRM250822C265,0.6080,0.0200,0.2716,-0.1453,27.28


⏳ Starting Phase 6: Freeze, Drift, Archive...
🆕 New trades: 0
📤 Closed trades: 0

🧾 Phase 6 Summary:
Total trades in master: 8
🧊 Frozen column null counts:
Delta_Entry            1
Gamma_Entry            1
Vega_Entry             1
Theta_Entry            1
IV_Entry               1
Skew_Entry             1
Kurtosis_Entry         1
IVHV_Gap_Entry         1
PCS_Entry              1
Confidence_Entry       1
Premium_Entry          1
CostBasis_Entry        1
Entry_Price            1
Time_Val_Entry         1
Intrinsic_Val_Entry    1
Premium                1
Spread                 1
BreakEven              1
Days_to_Earnings       1
Entry_Timestamp        0
Strategy               0
Structure              0
Tag_Intent             0
Tag_ExitStyle          0
Tag_EdgeType           0
Tag_IVHV_Tier          0
Is_Event_Setup         0
TradeDate              0
dtype: int64
✅ Phase 6 freeze and archive complete.

🧮 Days_Held injected for 7 rows
💹 Held_ROI% injected from % Total G/L
✅ Saved updated DataF

/Users/haniabadi/Documents/Github/options/core/phase6_freeze_and_archive.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2025-07-25T10:40:31.811336' '2025-07-28T14:41:48.029582'
 '2025-07-28T14:41:48.029582' '2025-07-28T14:41:48.029582'
 '2025-07-28T14:41:48.029582' '2025-07-28T14:41:48.029582'
 '2025-07-28T14:41:48.029582']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_existing.loc[mask, col] = df_existing.loc[mask, col_m]
/Users/haniabadi/Documents/Github/options/core/phase6_freeze_and_archive.py:114: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  today - pd.to_datetime(df_master_updated["TradeDate"], errors="coerce")
/Users/haniabadi/Documents/Github/options/core/phase6_5.py:10: UserWarning: Could not infer fo

In [ ]:
# # === ✅ Audit Phase 3.5 vs Master Integrity ===
# import pandas as pd
# from IPython.display import display

# def audit_35_to_master(df_35, df_master):
#     tracked_fields = [
#         "TradeID", "Strategy", "Structure", "PCS", "PCS_Entry",
#         "Delta_Entry", "Gamma_Entry", "Theta_Entry", "Vega_Entry",
#         "Premium_Entry", "IVHV_Gap_Entry", "Skew_Entry", "Kurtosis_Entry",
#         "Tag_Intent", "Tag_ExitStyle", "Confidence_Entry", "Confidence Tier",
#         "BreakEven", "Capital Deployed", "Margin Required", "CostBasis_Entry",
#         "Time_Val_Entry", "Intrinsic_Val_Entry", "TradeDate",
#         "Earnings_Date", "Days_to_Earnings", "Is_Event_Setup"
#     ]

#     results = []
#     for col in sorted(set(tracked_fields)):
#         results.append({
#             "Field": col,
#             "In Phase 3.5": col in df_35.columns,
#             "In Master": col in df_master.columns,
#             "Nulls in Master": df_master[col].isnull().sum() if col in df_master.columns else None,
#             "% Null in Master": round(df_master[col].isnull().mean() * 100, 2) if col in df_master.columns else None
#         })

#     audit_df = pd.DataFrame(results).sort_values("Field").reset_index(drop=True)
#     display(audit_df)
#     audit_df.to_csv("audit_35_to_master.csv", index=False)
#     return audit_df

# # Call the audit with current pipeline data
# audit_df = audit_35_to_master(df, df_master)

# df[df["BreakEven"].isnull()][["TradeID", "Strategy", "Strike", "Premium", "Quantity", "Structure"]]

import glob
import os

def get_latest_file(prefix, folder="data"):
    files = sorted(glob.glob(os.path.join(folder, f"{prefix}_*.csv")))
    return files[-1] if files else None


df_35 = pd.read_csv(get_latest_file("positions", folder="data"))
df_master = pd.read_csv(get_latest_file("master_with_drift", folder="data"))

# Track key drift fields
drift_fields = [
    "Delta_Drift", "Gamma_Drift", "Vega_Drift", "Theta_Drift", "PCS_Drift",
    "Delta_1D", "Gamma_3D", "Vega_5D",
    "Delta_1D_SMA", "Gamma_3D_SMA", "Vega_5D_SMA",
    "Delta_ROC", "Vega_ROC", "PCS_ROC", "Theta_ROC",
    "Delta_ROC_percent", "Gamma_ROC_percent", "Vega_ROC_percent"
]

audit = []
for col in drift_fields:
    audit.append({
        "Field": col,
        "In Master": col in df_master.columns,
        "In Drift Snapshot": col in df_drift_snapshot.columns,
        "Nulls in Master": df_master[col].isnull().sum() if col in df_master.columns else None
    })

pd.DataFrame(audit).sort_values("Field")


In [6]:
from core.phase1_clean import phase1_load_and_clean_raw_v2
from core.phase2_parse import phase2_parse_symbols, phase21_strategy_tagging

df, _ = phase1_load_and_clean_raw_v2()
df = phase2_parse_symbols(df)
df = phase21_strategy_tagging(df)

df.groupby(["Underlying", "Expiration", "OptionType"])["Strike"].nunique().sort_values()


⏳ Loading file: /Users/haniabadi/Documents/Windows/Positions_Account_.csv
🟢 File loaded!
🧹 Deduplicated 0 rows (Symbol only — fallback)
✅ Phase 1 complete.
📊 Rows: 7, Columns: 38
💾 Saved snapshot: /Users/haniabadi/Documents/Windows/Optionrec/drift/master_with_drift_2025-07-28_19-00-32.csv
⏱ Duration: 0.02s
✅ Phase 2.0 complete: OCC symbols parsed
✅ Phase 2.1 complete: Strategy tagging with TradeID-only logic


Underlying  Expiration  OptionType
BKNG        2025-08-01  Call          1
                        Put           1
CAT         2025-08-01  Call          1
CRM         2025-08-22  Call          1
                        Put           1
PLTR        2025-08-22  Call          1
                        Put           1
Name: Strike, dtype: int64

In [ ]:
import yfinance as yf
from core.chart_engine import aggregate_chart_tags

# Download 6+ months of data for any ticker
df = yf.Ticker("AMZN").history(start="2023-01-01", interval="1d")
df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()
df = df.astype("float64")
df.reset_index(inplace=True)

# Run your chart engine
chart_output = aggregate_chart_tags(df)

# View results
import pandas as pd
pd.Series(chart_output).sort_index()


In [ ]:
from core.phase7_drift_engine import run_phase7_drift_engine
run_phase7_drift_engine(update_master=True)


In [ ]:
from core.phase7_drift_engine import run_phase7_drift_engine

drift_dir = "/Users/haniabadi/Documents/Windows/Optionrec/drift"
df_drift = run_phase7_drift_engine(drift_dir, export_csv="drift_audit_latest.csv")

df_drift[df_drift["Flag_PCS_Drift"] | df_drift["Flag_Gamma_Collapse"]].tail(10)

In [ ]:
# from core.phase6_freeze_and_archive import phase6_freeze_and_archive
# df_master = phase6_freeze_and_archive(df)


import os
print("File exists:", os.path.exists("core/phase6_5.py"))


In [ ]:
# import pandas as pd

# pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv").columns

In [ ]:
import pandas as pd

# === Load master file and isolate a test ticker ===
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df_master = pd.read_csv(master_path)

test_ticker = "BKNG"  # Change as needed
df_single = df_master[df_master["Underlying"] == test_ticker].copy()

# === Debug: Check row count and required columns ===
print("Rows found:", df_single.shape[0])
required = ["PCS", "PCS_Drift", "Vega", "Gamma", "Delta", "Theta"]
missing = [col for col in required if col not in df_single.columns]
if missing:
    print("Missing columns:", missing)
else:
    print("✅ All required columns present.")

# === Force a test case if needed ===
if df_single.shape[0] > 0:
    i = df_single.index[0]
    df_single.loc[i, "PCS"] = 58
    df_single.loc[i, "PCS_Drift"] = 20
    df_single.loc[i, "Vega"] = 0.15
    df_single.loc[i, "Gamma"] = 0.01
    df_single.loc[i, "Delta"] = 0.30
    df_single.loc[i, "Theta"] = 0.35

# === PCS Engine v2 logic ===
def pcs_engine_v2(df):
    df = df.copy()
    df["PCS_Recalc"] = df["PCS"]

    def recommend(row):
        if row.get("PCS_Drift", 0) > 15 or row.get("PCS", 0) < 65:
            return "EXIT"
        elif row.get("PCS", 0) < 70 or row.get("Vega", 0) < 0.25:
            return "REVALIDATE"
        elif row.get("Gamma", 0) < 0.02:
            return "TRIM"
        return "HOLD"

    df["Rec_Action"] = df.apply(recommend, axis=1)

    def rationale(row):
        if row["Rec_Action"] == "EXIT":
            return "PCS < 65 or severe drift"
        elif row["Rec_Action"] == "REVALIDATE":
            return "PCS borderline or Vega < 0.25"
        elif row["Rec_Action"] == "TRIM":
            return "Gamma collapsed"
        return "Setup intact"

    df["Rec_Rationale"] = df.apply(rationale, axis=1)

    def tier(row):
        return {"EXIT": 1, "REVALIDATE": 2, "TRIM": 3, "HOLD": 4}.get(row["Rec_Action"], 4)

    df["Rec_Tier"] = df.apply(tier, axis=1)

    df["Greek_Conflict"] = (
        ((df["Delta"] < 0.35) & (df["Gamma"] < 0.03)) |
        ((df["Theta"] > df["Vega"]) & (df["Vega"] < 0.2))
    )

    df["Repair_Trigger"] = (
        (df["PCS"] < 70) & (df["Vega"] < 0.25) & (df["Gamma"] < 0.03)
    )

    df["Leg_Divergence_Score"] = abs(df.get("Call_PCS", 0) - df.get("Put_PCS", 0))
    return df

# === Run the engine and display results ===
df_result = pcs_engine_v2(df_single)

# === Display full result no matter what ===
pd.set_option("display.max_columns", None)
display(df_result[[
    "TradeID", "Underlying", "PCS", "PCS_Drift", "Vega", "Gamma", "Delta", "Theta",
    "Rec_Action", "Rec_Rationale", "Rec_Tier", "Greek_Conflict", "Repair_Trigger"
]])

print(df_master["Underlying"].unique())


In [ ]:
import sys
import pandas as pd

# ✅ Step 1: Add the core folder to Python's path
sys.path.append("./core")  # or full path if needed

# ✅ Step 2: Import the function
from pcs_engine_v3_unified import pcs_engine_v3_unified

# ✅ Step 3: Load your master file
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df_master = pd.read_csv(master_path)

# ✅ Step 4: Run the engine
df_result = pcs_engine_v3_unified(df_master)

# ✅ Step 5: Display result
df_result[[
    "TradeID", "Underlying", "PCS", "PCS_UnifiedScore", "PCS_SignalScore",
    "Chart_Support", "Rec_Action", "Rec_Tier", "Trade_Health_Tier",
    "Rationale_Composite", "Persona_Violation", "Recovery_Bias"
]].sort_values("Rec_Tier")


In [ ]:
# import yfinance as yf
# from core.chart_engine import aggregate_chart_tags

# # Download 6+ months of data for any ticker
# df = yf.Ticker("AMZN").history(start="2023-01-01", interval="1d")
# df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()
# df = df.astype("float64")
# df.reset_index(inplace=True)

# # Run your chart engine
# chart_output = aggregate_chart_tags(df)

# # View results
# import pandas as pd
# pd.Series(chart_output).sort_index()


# from core.chart_engine import run_phase8_chart_engine

# run_phase8_chart_engine(master_path="/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv")


In [ ]:
def build_rec_debug(row):
    lines = []
    lines.append(f"PCS: {row['PCS']}, Drift: {row.get('PCS_Drift', 'NA')}, Vega: {row['Vega']}, Gamma: {row['Gamma']}")
    lines.append(f"UnifiedScore: {row['PCS_UnifiedScore']:.2f}, SignalScore: {row['PCS_SignalScore']:.2f}, PersonaScore: {row['PCS_PersonaScore']:.1f}")
    lines.append(f"Strategy: {row['Strategy_Tier']} | Chart_Support: {row['Chart_Support']} | Match_Rank: {row['Strategy_Match_Rank']}")
    if row['Persona_Violation']:
        lines.append("⚠️ Persona Violation")
    if row['Recovery_Bias']:
        lines.append("🟢 Recovery Bias")
    if row['PCS_UnifiedScore'] < 60:
        lines.append("⛔ UnifiedScore < 60 → EXIT")
    elif row.get('PCS_Drift', 0) > 20:
        lines.append("📉 PCS Drift > 20 → EXIT")
    elif not row['Chart_Support']:
        lines.append("🟥 Chart Breakdown → REVALIDATE")
    elif row['PCS_UnifiedScore'] < 70:
        lines.append("⚠️ Score < 70 → TRIM")
    else:
        lines.append("✅ Holding Edge Intact")
    return " | ".join(lines)

# Apply to your result DataFrame
df_result["Rec_Debug"] = df_result.apply(build_rec_debug, axis=1)

# Preview full decision path
df_result[[
    "TradeID", "PCS", "PCS_UnifiedScore", "Rec_Action", "Rec_Tier", "Rationale_Composite", "Rec_Debug"
]].sort_values("Rec_Tier")


In [ ]:
import sys
import pandas as pd

sys.path.append("core")
from tradier_chain import get_tradier_greeks

# Pull chain
df_chain = get_tradier_greeks("AAPL", "2025-08-15")

# 🔧 Flatten greeks if needed
if "greeks" in df_chain.columns:
    greeks_df = pd.json_normalize(df_chain["greeks"])
    df_chain = pd.concat([df_chain.drop(columns=["greeks"]), greeks_df], axis=1)

# 🎯 Display useful columns
cols = [
    "symbol", "strike", "option_type", "expiration_date",
    "delta", "gamma", "vega", "theta", "implied_volatility",
    "bid", "ask", "last", "volume", "open_interest"
]
df_chain[[c for c in cols if c in df_chain.columns]].head(10)


In [ ]:
# === Phase 1: Load & Clean Raw Trades
from core.phase1_clean import phase1_load_and_clean_raw_v2
df, _ = phase1_load_and_clean_raw_v2()

# === Phase 2: Parse OCC Symbol + Tag Strategy
from core.phase2_parse import phase2_parse_symbols, phase21_strategy_tagging
df = phase2_parse_symbols(df)
df = phase21_strategy_tagging(df)

# === Phase 3: Calculate PCS + Greeks
from core.phase3_pcs_score import calculate_ivhv_gap, calculate_skew_and_kurtosis, calculate_pcs
df = calculate_ivhv_gap(df)
df = calculate_skew_and_kurtosis(df)
df = calculate_pcs(df)

# === Phase 3.5: Add Freeze Fields
from core.phase3_5_freeze_fields import phase35_fill_freeze_fields
df = phase35_fill_freeze_fields(df)

# === Phase 4: Save Snapshot
from core.phase4_snapshot import phase4_save_snapshot
_, snapshot_path = phase4_save_snapshot(df)

# === Phase 5/6: Load Master + Freeze Archive
from utils.load_master_snapshot import load_master_snapshot
from core.phase6_freeze_and_archive import phase6_freeze_and_archive
master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df_master_current = load_master_snapshot(master_path)
df_master = phase6_freeze_and_archive(df, df_master_current)

# === Phase 6.5: ROI, Days Held, etc.
from core.phase6_5 import phase6_5_inject_derived_fields
df_master = phase6_5_inject_derived_fields(df_master, save_path=master_path)

# === Phase 7: Drift Engine
from core.phase7_drift_engine import run_phase7_drift_engine
drift_dir = "/Users/haniabadi/Documents/Windows/Optionrec/drift"
df_drift = run_phase7_drift_engine(drift_dir=drift_dir, update_master=True)

# === Phase 8: Chart Engine
from core.chart_engine import run_phase8_chart_engine
run_phase8_chart_engine(master_path=master_path)

# === Final Revalidation (PCS Engine v3.2)
from pcs_engine_v3_2 import pcs_engine_v3_2_strategy_aware
df_result = pcs_engine_v3_2_strategy_aware(df_master)

# === Rec_Debug Column
def build_rec_debug(row):
    lines = []
    lines.append(f"PCS: {row['PCS']}, Drift: {row.get('PCS_Drift', 'NA')}, Vega: {row['Vega']}, Gamma: {row['Gamma']}")
    lines.append(f"UnifiedScore: {row['PCS_UnifiedScore']:.2f}, SignalScore: {row['PCS_SignalScore']:.2f}, PersonaScore: {row.get('PCS_PersonaScore', 0):.1f}")
    lines.append(f"Strategy: {row.get('Strategy_Tier')} | Chart_Support: {row['Chart_Support']} | Match_Rank: {row.get('Strategy_Match_Rank')}")
    if row.get('Persona_Violation'): lines.append("⚠️ Persona Violation")
    if row.get('Recovery_Bias'): lines.append("🟢 Recovery Bias")
    if row.get('PCS_UnifiedScore', 0) < 60:
        lines.append("⛔ UnifiedScore < 60 → EXIT")
    elif row.get('PCS_Drift', 0) > 20:
        lines.append("📉 PCS Drift > 20 → EXIT")
    elif not row.get('Chart_Support', True):
        lines.append("🟥 Chart Breakdown → REVALIDATE")
    elif row.get('PCS_UnifiedScore', 0) < 70:
        lines.append("⚠️ Score < 70 → TRIM")
    else:
        lines.append("✅ Holding Edge Intact")
    return " | ".join(lines)

df_result["Rec_Debug"] = df_result.apply(build_rec_debug, axis=1)

# === Display Sorted Output
df_result[[
    "TradeID", "Underlying", "PCS", "PCS_UnifiedScore", "PCS_PersonaScore",
    "Rec_Action", "Rec_Tier", "Strategy_Tier", "Trade_Health_Tier", "Rationale_Composite", "Rec_Debug"
]].sort_values("Rec_Tier")


In [ ]:
from core.phase1_clean import phase1_load_and_clean_raw_v2
df, _ = phase1_load_and_clean_raw_v2()
print(df[["Symbol"]])
print("🧩 Rows after Phase 1:", len(df))


In [ ]:
from core.phase2_parse import phase2_parse_symbols
df = phase2_parse_symbols(df)
print(df[["Symbol", "OptionType", "Strike", "Expiration"]])
print("🧩 Rows after Phase 2.0:", len(df))


In [ ]:
from core.phase2_parse import unify_trade_ids
df = unify_trade_ids(df)

print(df[["Symbol", "TradeID", "OptionType", "Strike"]])
print("🧩 Unique TradeIDs:", df["TradeID"].nunique())


In [ ]:
from core.phase2_parse import phase21_strategy_tagging
df = phase21_strategy_tagging(df, debug=True)

df[["TradeID", "Symbol", "Strategy", "Type"]]


In [ ]:
from core.phase3_pcs_score import calculate_ivhv_gap, calculate_skew_and_kurtosis, calculate_pcs
from core.phase3_5_freeze_fields import phase3_5_fill_freeze_fields

df = calculate_ivhv_gap(df)
df = calculate_skew_and_kurtosis(df)
df = calculate_pcs(df)
df = phase3_5_fill_freeze_fields(df)

print(df[["TradeID", "Strategy", "PCS", "Skew_Entry", "BreakEven", "Tag_ExitStyle", "Tag_Intent"]])


In [ ]:
from core.phase4_snapshot import phase4_save_snapshot
df, snapshot_path = phase4_save_snapshot(df)
print(snapshot_path)


In [ ]:
from utils.load_master_snapshot import load_master_snapshot
from core.phase6_freeze_and_archive import phase6_freeze_and_archive

master_path = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
df_master_current = load_master_snapshot(master_path)
df_master = phase6_freeze_and_archive(df, df_master_current)

df_master[["TradeID", "Strategy", "PCS_Entry", "BreakEven", "Delta_Entry", "Held_ROI%", "Tag_ExitStyle"]]


In [ ]:
from core.phase6_5 import phase6_5_inject_derived_fields
df_master = phase6_5_inject_derived_fields(df_master, save_path=master_path)

# Then safely print:
df_master[["TradeID", "Strategy", "PCS_Entry", "BreakEven", "Delta_Entry", "Held_ROI%", "Tag_ExitStyle"]]


In [ ]:
df["TradeID"].value_counts()
df_master["TradeID"].value_counts()


In [ ]:
from core.phase3_5_freeze_fields import phase3_5_fill_freeze_fields
df = phase3_5_fill_freeze_fields(df)

?evaluate_leg_status(df, legs_dir="/Optionrec/legs")


In [ ]:
df_master = pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv")
print(df_master["TradeID"].unique()[:10])
